# Intro
This notebook can process many PDFs in parallel (with two-pass "Deep Parsing" to generate image descriptions). Upload PDFs in bulk to the pdfs directory and run the cells. The setup notebook needs to have been run first to prep the environment.

# Cell 0 Unload GPU (Optional)
- Used between runs to free GPU VRAM if needed/wanted

In [ ]:
# Cell 0: Clear GPU Memory
import gc
import torch

print("="*70)
print("CLEARING GPU MEMORY")
print("="*70)

# Delete model if it exists
if 'llm' in globals():
    print("✓ Unloading model...")
    del llm

# Delete any other large objects that might be in memory
vars_to_clear = ['processor', 'pass2_processor', 'outputs', 'batch_inputs', 
                 'images', 'original_images', 'figure_descriptions']
for var_name in vars_to_clear:
    if var_name in globals():
        print(f"✓ Clearing {var_name}...")
        del globals()[var_name]

# Force garbage collection
gc.collect()

# Clear CUDA cache
torch.cuda.empty_cache()

# Force synchronize to ensure cleanup is complete
if torch.cuda.is_available():
    torch.cuda.synchronize()

# Show results
print(f"\nGPU Memory Status:")
print(f"  Allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
print(f"  Reserved: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")
print(f"  Free: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)) / 1024**3:.2f} GB")

print("\n✓ GPU memory cleared - ready for clean run")
print("="*70)

# Cell 1: Validate Environment
- Confirm everything is installed and directories are in place as expected
- Typically run only once

In [ ]:
# Cell 1: Validate Environment
import torch
import vllm
import os
import sys
from pathlib import Path
import random
import time

print("="*70)
print("VALIDATING ENVIRONMENT")
print("="*70)

# Verify versions
print(f"\nvLLM version: {vllm.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA version: {torch.version.cuda}")
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

assert vllm.__version__ == "0.8.5", "Wrong vLLM version! Run setup notebook."

# Setup paths
MODEL_PATH = "/workspace/model/DeepSeek-OCR"
PDF_INPUT_DIR = "/workspace/pdfs"
TEMP_DIR = "/workspace/temp"
TEMP_PAGES_DIR = "/workspace/temp/pages"
TEMP_FIGURES_DIR = "/workspace/temp/figures"
TEMP_PASS1_RAW_DIR = "/workspace/temp/pass1_raw"
OUTPUT_DIR = "/workspace/output"
REPORTS_DIR = "/workspace/reports"

# Create all directories
for dir_path in [TEMP_DIR, TEMP_PAGES_DIR, TEMP_FIGURES_DIR, 
                 TEMP_PASS1_RAW_DIR, OUTPUT_DIR, REPORTS_DIR]:
    os.makedirs(dir_path, exist_ok=True)

print(f"\nDirectories:")
print(f"  Model: {MODEL_PATH}")
print(f"  PDFs: {PDF_INPUT_DIR}")
print(f"  Temp (pages): {TEMP_PAGES_DIR}")
print(f"  Temp (figures): {TEMP_FIGURES_DIR}")
print(f"  Output: {OUTPUT_DIR}")
print(f"  Reports: {REPORTS_DIR}")
print("\n✓ Environment Validated!")

# Cell 2: Cleanup Temp Directories
- Remove any leftover files from previous runs
- Pages directory holds images of pages from PDFs, figures directory holds figures that have been extracted from pages, PASS1_RAW_DIR holds raw markdown output after pass 1 (including markdown tags and image coordinates)

In [ ]:
# Cell 2: Cleanup Temp Directories
import shutil
from pathlib import Path

print("="*70)
print("CLEANING TEMP DIRECTORIES")
print("="*70)

temp_dirs = [TEMP_PAGES_DIR, TEMP_FIGURES_DIR, TEMP_PASS1_RAW_DIR]

for temp_dir in temp_dirs:
    if Path(temp_dir).exists():
        file_count = len(list(Path(temp_dir).glob("*")))
        shutil.rmtree(temp_dir)
        os.makedirs(temp_dir, exist_ok=True)
        print(f"✓ Cleaned {temp_dir}")
        print(f"  Removed {file_count} files")
    else:
        print(f"⚠ {temp_dir} doesn't exist")

print("\n✓ All temp directories cleaned and ready for next run")
print("="*70)

# Cell 3a: Pass 1 Configuration
- Pass 1 converts a PDF to markdown but does not produce images - it only identifies their locations.
- Override config.py (in the cloned repo) to provide control over key settings here in the notebook. Adjust NUM_WORKERS to control parallelization of CPU operations.

In [ ]:
# Cell 3a: Configuration - Pass 1
import subprocess
import psutil

print("="*70)
print("CONFIGURATION - PASS 1")
print("="*70)

# Add DeepSeek repo to path
repo_path = '/workspace/DeepSeek-OCR-repo'
vllm_impl_path = os.path.join(repo_path, 'DeepSeek-OCR-master', 'DeepSeek-OCR-vllm')

if not os.path.exists(repo_path):
    print("\n⚠ Repository not found, cloning...")
    subprocess.run(['git', 'clone', 'https://github.com/deepseek-ai/DeepSeek-OCR.git', repo_path], 
                   check=True, capture_output=True)
    print("✓ Repository cloned")

if vllm_impl_path not in sys.path:
    sys.path.insert(0, vllm_impl_path)

# Import config module (but don't import processor yet!)
import config

# CPU Configuration
physical_cores = psutil.cpu_count(logical=False)
logical_cores = psutil.cpu_count(logical=True)

cpu_model = subprocess.run(
    ["grep", "-m", "1", "model name", "/proc/cpuinfo"],
    capture_output=True,
    text=True
).stdout.strip().split(':')[1].strip()

print(f"\nCPU Info:")
print(f"  Model: {cpu_model}")
print(f"  Physical Cores: {physical_cores}")
print(f"  Logical Cores: {logical_cores}")

# ============================================================
# PASS 1 SETTINGS - USER CONFIGURABLE
# ============================================================

# PDF to Image Conversion
PDF_DPI = 144  # DPI for PDF page images (higher = better quality, larger files)

# Image Processing Settings
BASE_SIZE = 1024
IMAGE_SIZE = 640
CROP_MODE = True
MIN_CROPS = 2
MAX_CROPS = 6
NUM_WORKERS = 32 # physical_cores  # CPU preprocessing threads

# GPU Model Settings
PASS1_MAX_MODEL_LEN = 8192
PASS1_MAX_NUM_SEQS = 100  # Batch size / controls number of images processed in parallel
PASS1_GPU_MEMORY_UTIL = 0.95

# Generation Settings
PASS1_TEMPERATURE = 0.0
PASS1_MAX_TOKENS = 8192

# Prompt
PASS1_PROMPT = '<image>\n<|grounding|>Convert the document to markdown.'

# ============================================================

# Override config.py with our settings
config.BASE_SIZE = BASE_SIZE
config.IMAGE_SIZE = IMAGE_SIZE
config.CROP_MODE = CROP_MODE
config.MIN_CROPS = MIN_CROPS
config.MAX_CROPS = MAX_CROPS
config.NUM_WORKERS = NUM_WORKERS
config.MAX_CONCURRENCY = PASS1_MAX_NUM_SEQS
config.MODEL_PATH = MODEL_PATH
config.PROMPT = PASS1_PROMPT

print(f"\n✓ Configuration Override Applied")
print(f"  BASE_SIZE: {BASE_SIZE}")
print(f"  IMAGE_SIZE: {IMAGE_SIZE}")
print(f"  CROP_MODE: {CROP_MODE}")
print(f"  MIN_CROPS: {MIN_CROPS}")
print(f"  MAX_CROPS: {MAX_CROPS}")
print(f"  Expected tokens per image: 64")

print(f"\nCPU Preprocessing:")
print(f"  NUM_WORKERS: {NUM_WORKERS}")
print(f"  💡 Recommended range: {int(physical_cores * 1.5)}-{physical_cores * 2}")

print(f"\nGPU Model Settings:")
print(f"  max_model_len: {PASS1_MAX_MODEL_LEN}")
print(f"  max_num_seqs (batch size): {PASS1_MAX_NUM_SEQS}")
print(f"  gpu_memory_utilization: {PASS1_GPU_MEMORY_UTIL}")

print(f"\nGeneration Settings:")
print(f"  temperature: {PASS1_TEMPERATURE}")
print(f"  max_tokens: {PASS1_MAX_TOKENS}")

print(f"\nPrompt:")
print(f"  {PASS1_PROMPT}")

print("\n✓ Configuration complete")

# Cell 3b: Pass 2 Configuration
- Pass 2 uses image locations identified in pass 1, extracts and saves the images, then runs the images through the model with a different prompt to produce image descriptions
- This cell (like 3a) also overrides config.py (in the cloned repo) to provide control over key settings here in the notebook. Pay attention to CPU NUM_WORKERS.
- Higher batch size (200 vs 100) - cropped figures are smaller, can fit more in VRAM
- Higher temperature (0.2 vs 0.0) - encourages more natural descriptions
- Fewer max_tokens (2048 vs 8192) - descriptions are shorter than full-page OCR

In [ ]:
# Cell 3b: Configuration - Pass 2
print("="*70)
print("CONFIGURATION - PASS 2")
print("="*70)

# ============================================================
# PASS 2 SETTINGS - USER CONFIGURABLE
# ============================================================

# Image Processing Settings (reuse from Pass 1, or customize)
PASS2_BASE_SIZE = 1024  # Can be same or different from Pass 1
PASS2_IMAGE_SIZE = 640
PASS2_CROP_MODE = True
PASS2_MIN_CROPS = 2
PASS2_MAX_CROPS = 6
PASS2_NUM_WORKERS = 32 # physical_cores  # CPU preprocessing threads

# GPU Model Settings
PASS2_MAX_MODEL_LEN = 8192
PASS2_MAX_NUM_SEQS = 200  # Higher than Pass 1 - cropped figures are smaller!
PASS2_GPU_MEMORY_UTIL = 0.95

# Generation Settings
PASS2_TEMPERATURE = 0.2  # Slightly higher for more natural descriptions
PASS2_MAX_TOKENS = 2048  # Descriptions are shorter than full page OCR

# Prompt
PASS2_PROMPT = '<image>\nDescribe this image in detail'

# ============================================================

print(f"\n✓ Pass 2 Configuration Set")

print(f"\nImage Processing:")
print(f"  BASE_SIZE: {PASS2_BASE_SIZE}")
print(f"  IMAGE_SIZE: {PASS2_IMAGE_SIZE}")
print(f"  CROP_MODE: {PASS2_CROP_MODE}")
print(f"  NUM_WORKERS: {PASS2_NUM_WORKERS}")

print(f"\nGPU Model Settings:")
print(f"  max_model_len: {PASS2_MAX_MODEL_LEN}")
print(f"  max_num_seqs (batch size): {PASS2_MAX_NUM_SEQS}")
print(f"  💡 Increased from Pass 1 ({PASS1_MAX_NUM_SEQS}) - figures are smaller")
print(f"  gpu_memory_utilization: {PASS2_GPU_MEMORY_UTIL}")

print(f"\nGeneration Settings:")
print(f"  temperature: {PASS2_TEMPERATURE} (higher for natural language)")
print(f"  max_tokens: {PASS2_MAX_TOKENS} (shorter than Pass 1)")

print(f"\nPrompt:")
print(f"  {PASS2_PROMPT}")

print("\n✓ Pass 2 configuration complete")
print("="*70)

# Cell 4: PDF Inventory
- This cell determines the number of pdfs in the `pdfs` directory, the file size for the job, and creates a list of files to be pre-processed

In [ ]:
# Cell 4: PDF Inventory
from pathlib import Path

pdf_files = sorted(Path(PDF_INPUT_DIR).glob("*.pdf"))

print("="*70)
print("PDF INVENTORY")
print("="*70)

if len(pdf_files) == 0:
    raise FileNotFoundError(f"No PDFs found in {PDF_INPUT_DIR}")

total_size_mb = sum(f.stat().st_size for f in pdf_files) / (1024**2)
total_size_gb = total_size_mb / 1024

print(f"\nFound {len(pdf_files)} PDF(s)")
print(f"Total size: {total_size_mb:.2f} MB ({total_size_gb:.3f} GB)")
print(f"\nFiles:")
for pdf in pdf_files:
    size_mb = pdf.stat().st_size / (1024**2)
    print(f"  - {pdf.name} ({size_mb:.2f} MB)")

print("\n✓ Ready for preprocessing")

# Cell 5: Load Model for Pass 1
- This cell loads DeepseekOCRForCausalLM which is actually the entire pipeline in one unified model object
- This includes:
    - DeepEncoder (Vision Processing)
        - SAM (Segment Anything Model) - 80M params
        - 16× Convolutional Compressor
        - CLIP-large - 300M params
    - DeepSeek-3B-MoE (Language Decoder)
        - Mixture of Experts decoder - generates text
- This is all contained in `deepseek_ocr.py` in the GitHub repo

In [ ]:
# Cell 5: Load Model
import gc
import torch

# Clear any existing model from memory
if 'llm' in globals():
    del llm
gc.collect()
torch.cuda.empty_cache()

print("="*70)
print("LOADING MODEL")
print("="*70)

os.environ['VLLM_USE_V1'] = '0'

# Import and register (repo already in path from Cell 2)
from deepseek_ocr import DeepseekOCRForCausalLM
from vllm.model_executor.models.registry import ModelRegistry
from vllm import LLM, SamplingParams
from process.ngram_norepeat import NoRepeatNGramLogitsProcessor

ModelRegistry.register_model("DeepseekOCRForCausalLM", DeepseekOCRForCausalLM)

print(f"\nGPU Memory Before Load: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")

llm = LLM(
    model=MODEL_PATH,
    hf_overrides={"architectures": ["DeepseekOCRForCausalLM"]},
    block_size=256,
    enforce_eager=False,
    trust_remote_code=True, 
    max_model_len=PASS1_MAX_MODEL_LEN,
    swap_space=0,
    max_num_seqs=PASS1_MAX_NUM_SEQS,
    tensor_parallel_size=1,
    gpu_memory_utilization=PASS1_GPU_MEMORY_UTIL,
    disable_mm_preprocessor_cache=True
)

print(f"GPU Memory After Load: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")

# Setup sampling parameters for Pass 1
logits_processors = [NoRepeatNGramLogitsProcessor(
    ngram_size=20, window_size=50, whitelist_token_ids={128821, 128822}
)]

sampling_params_pass1 = SamplingParams(
    temperature=PASS1_TEMPERATURE,
    max_tokens=PASS1_MAX_TOKENS,
    logits_processors=logits_processors,
    skip_special_tokens=False,
    include_stop_str_in_output=True,
)

print("\n✓ Model loaded and ready!")

# Cell 6a: Pass 1 CPU Preprocessing - Convert All PDFs to Images for Each Page
- The DeepSeek pipeline uses CPU to pre-process the PDFs by converting each page to an image
- Overall job timing starts in this cell
- This cell parallelizes PDF pages across NUM_WORKERS. You can adjust this in an earlier cell based on your hardware
- Saves PNG images of each page to temp/pages

In [ ]:
# Cell 6a: CPU Preprocessing - Convert All PDFs to Page Images (Parallelized)
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger().setLevel(logging.ERROR)

import time
import fitz
from PIL import Image
import io as io_module
from pathlib import Path
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed

def convert_single_page(pdf_path, page_num, dpi=None):
    """Convert a single page to an image and save it"""
    if dpi is None:
        dpi = PDF_DPI  # Use config value
    try:
        pdf_document = fitz.open(pdf_path)
        zoom = dpi / 72.0
        matrix = fitz.Matrix(zoom, zoom)
        
        page = pdf_document[page_num]
        pixmap = page.get_pixmap(matrix=matrix, alpha=False)
        img_data = pixmap.tobytes("png")
        img = Image.open(io_module.BytesIO(img_data))
        
        # Save immediately
        pdf_stem = Path(pdf_path).stem
        img_filename = f"{pdf_stem}_page{page_num+1:04d}.png"
        img_path = Path(TEMP_PAGES_DIR) / img_filename
        img.save(img_path)
        
        pdf_document.close()
        return (pdf_path, page_num, True, img_filename)
    except Exception as e:
        return (pdf_path, page_num, False, str(e))

print("="*70)
print("CPU PREPROCESSING: PDF → PAGE IMAGES (PARALLELIZED)")
print("="*70)

# Initialize timing tracker
timing_data = {}
pipeline_start = time.time()

# Clear temp/pages directory
if os.path.exists(TEMP_PAGES_DIR):
    shutil.rmtree(TEMP_PAGES_DIR)
os.makedirs(TEMP_PAGES_DIR, exist_ok=True)

preprocessing_start = time.time()

# Build list of all pages to process
page_jobs = []
for pdf_path in pdf_files:
    try:
        pdf_document = fitz.open(str(pdf_path))
        num_pages = pdf_document.page_count
        pdf_document.close()
        
        for page_num in range(num_pages):
            page_jobs.append((str(pdf_path), page_num))
        
        print(f"Queued: {pdf_path.name} ({num_pages} pages)")
    except Exception as e:
        print(f"✗ Failed to open {pdf_path.name}: {e}")

total_pages = len(page_jobs)
print(f"\nTotal pages to process: {total_pages}")
print(f"Using {NUM_WORKERS} worker threads\n")

# Process all pages in parallel
successful = 0
failed = 0

with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
    # Submit all jobs
    futures = {
        executor.submit(convert_single_page, pdf_path, page_num): (pdf_path, page_num)
        for pdf_path, page_num in page_jobs
    }
    
    # Process as they complete
    for future in as_completed(futures):
        pdf_path, page_num, success, result = future.result()
        if success:
            successful += 1
        else:
            failed += 1
            print(f"✗ Failed: {Path(pdf_path).name} page {page_num+1}: {result}")
        
        # Progress indicator every 100 pages
        if (successful + failed) % 100 == 0:
            print(f"  Progress: {successful + failed}/{total_pages} pages processed...")

preprocessing_time = time.time() - preprocessing_start
pages_per_sec = successful / preprocessing_time if preprocessing_time > 0 else 0

print(f"\n{'='*70}")
print(f"✓ Preprocessing Complete")
print(f"  Total Pages: {total_pages}")
print(f"  Successful: {successful}")
print(f"  Failed: {failed}")
print(f"  Time: {preprocessing_time:.2f}s ({pages_per_sec:.2f} pages/sec)")
print(f"  Workers: {NUM_WORKERS}")
print(f"  Images saved to: {TEMP_PAGES_DIR}")
print(f"{'='*70}")

timing_data['pdf_to_images'] = preprocessing_time

# Cell 6b: Pass 1 CPU Preprocessing - Apply "DeepSeek Preprocessing"
- The DeepSeek pipeline uses CPU to resize, crop, tile, and tokenize the page images (depending on resolution mode) prior to GPU processing.
- This cell parallelizes PDF pages across NUM_WORKERS. You can adjust this in an earlier cell based on your hardware
- This creates `batch_inputs` which contains the prompt and tensors representing each page (list of dictionaries) and is stored in RAM for fast transfer into the GPU 

In [ ]:
# Cell 6b: CPU Preprocessing - Apply DeepSeek Image Processing
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger().setLevel(logging.ERROR)

import sys
import importlib
from PIL import Image
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

print("="*70)
print("CPU PREPROCESSING: DEEPSEEK IMAGE PROCESSING")
print("="*70)

# CRITICAL: Force reload config to Pass 1 settings before creating processor to avoid accidental use of Pass 2 settings
print("\n[Config] Forcing Pass 1 configuration reload...")
config_module = sys.modules['config']
config_module.PROMPT = PASS1_PROMPT
config_module.BASE_SIZE = BASE_SIZE
config_module.IMAGE_SIZE = IMAGE_SIZE
config_module.CROP_MODE = CROP_MODE
config_module.NUM_WORKERS = NUM_WORKERS

# Reload image_process module to pick up Pass 1 config
if 'process.image_process' in sys.modules:
    import process.image_process
    importlib.reload(process.image_process)

from process.image_process import DeepseekOCRProcessor

# Create NEW processor instance with Pass 1 settings
processor = DeepseekOCRProcessor()

print(f"✓ Processor created with Pass 1 configuration")
print(f"  BASE_SIZE: {config_module.BASE_SIZE}")
print(f"  IMAGE_SIZE: {config_module.IMAGE_SIZE}")
print(f"  CROP_MODE: {config_module.CROP_MODE}")
print(f"  NUM_WORKERS: {config_module.NUM_WORKERS}")
print(f"  PROMPT: {config_module.PROMPT}")

def process_single_page_image(img_path):
    """Apply DeepSeek preprocessing to a single page image"""
    try:
        # Load image
        image = Image.open(img_path)
        
        # Apply DeepSeek preprocessing (resize, crop, tile, tokenize)
        processed = processor.tokenize_with_images(
            images=[image], 
            bos=True, 
            eos=True, 
            cropping=config.CROP_MODE
        )
        
        return (img_path.name, True, processed)
    except Exception as e:
        return (img_path.name, False, str(e))

# Get all page images from temp/pages/ in SORTED order
page_image_files = sorted(Path(TEMP_PAGES_DIR).glob("*.png"))
total_images = len(page_image_files)

print(f"\nFound {total_images} page images to preprocess")
print(f"Using {NUM_WORKERS} worker threads\n")

preprocess_start = time.time()

# Store results with filenames for later sorting
results = []
successful = 0
failed = 0

with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
    # Submit all jobs
    futures = {
        executor.submit(process_single_page_image, img_path): img_path
        for img_path in page_image_files
    }
    
    # Collect results as they complete
    for future in as_completed(futures):
        img_name, success, result = future.result()
        
        if success:
            results.append((img_name, result))  # Store filename with result
            successful += 1
        else:
            failed += 1
            print(f"✗ Failed: {img_name}: {result}")
        
        # Progress indicator
        if (successful + failed) % 100 == 0:
            print(f"  Progress: {successful + failed}/{total_images} images processed...")

# CRITICAL: Sort results by filename to maintain order
results.sort(key=lambda x: x[0])

# Build batch_inputs in sorted order
batch_inputs = []
for img_name, processed in results:
    batch_inputs.append({
        "prompt": PASS1_PROMPT,
        "multi_modal_data": {"image": processed},
    })

preprocess_time = time.time() - preprocess_start
images_per_sec = successful / preprocess_time if preprocess_time > 0 else 0

print(f"\n{'='*70}")
print(f"  Total Images: {total_images}")
print(f"  Successful: {successful}")
print(f"  Failed: {failed}")
print(f"  Time: {preprocess_time:.2f}s ({images_per_sec:.2f} images/sec)")
print(f"  Workers: {NUM_WORKERS}")
print(f"  Batch inputs ready (SORTED): {len(batch_inputs)}")
print(f"✓ DeepSeek Preprocessing Complete")
print(f"{'='*70}")

timing_data['deepseek_preprocessing_pass1'] = preprocess_time

# Cell 6c: Run Pass 1 on GPU
- Clears temp/pass1_raw/ directory
- Takes the batch_inputs from previous cell (already preprocessed)
- Runs through GPU with llm.generate() (batched automatically by vLLM)
- Saves each page's markdown output to temp/pass1_raw/ with matching filename

In [ ]:
# Cell 6c: GPU Processing - Pass 1 OCR
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger().setLevel(logging.ERROR)

import time
from pathlib import Path
import shutil

print("="*70)
print("GPU PROCESSING - PASS 1 OCR")
print("="*70)

# Ensure config is set to Pass 1 (in case Pass 2 was run previously)
config.PROMPT = PASS1_PROMPT
print(f"✓ Config set to Pass 1 prompt")

# Clear pass1_raw directory
if os.path.exists(TEMP_PASS1_RAW_DIR):
    shutil.rmtree(TEMP_PASS1_RAW_DIR)
os.makedirs(TEMP_PASS1_RAW_DIR, exist_ok=True)

# Check that we have preprocessed batch_inputs from Cell 4b
if 'batch_inputs' not in globals() or len(batch_inputs) == 0:
    raise RuntimeError("No batch_inputs found! Run Cell 4b first.")

total_pages = len(batch_inputs)
print(f"\nProcessing {total_pages} preprocessed page images")
print(f"Batch size (max_num_seqs): {PASS1_MAX_NUM_SEQS}")
print(f"GPU Memory Utilization: {PASS1_GPU_MEMORY_UTIL}")

# Run GPU inference
gpu_start = time.time()

outputs = llm.generate(batch_inputs, sampling_params=sampling_params_pass1)

gpu_time = time.time() - gpu_start
pages_per_sec = total_pages / gpu_time if gpu_time > 0 else 0

print(f"\n✓ GPU Processing Complete")
print(f"  Time: {gpu_time:.2f}s ({pages_per_sec:.2f} pages/sec)")

# Save outputs to temp/pass1_raw/
print(f"\nSaving markdown outputs to {TEMP_PASS1_RAW_DIR}...")

# Get original filenames from temp/pages to maintain mapping
page_image_files = sorted(Path(TEMP_PAGES_DIR).glob("*.png"))

for img_path, output in zip(page_image_files, outputs):
    # Extract PDF name and page number from filename
    # Format: {pdf_stem}_page{0001}.png
    img_stem = img_path.stem  # e.g., "document_page0001"
    
    # Save markdown with same naming convention
    markdown_filename = f"{img_stem}.md"
    markdown_path = Path(TEMP_PASS1_RAW_DIR) / markdown_filename
    
    # Get generated text
    markdown_text = output.outputs[0].text
    
    # Save to file
    with open(markdown_path, 'w', encoding='utf-8') as f:
        f.write(markdown_text)

print(f"✓ Saved {len(outputs)} markdown files")

print(f"\n{'='*70}")
print(f"PASS 1 COMPLETE")
print(f"{'='*70}")
print(f"  Total Pages: {total_pages}")
print(f"  GPU Time: {gpu_time:.2f}s")
print(f"  Throughput: {pages_per_sec:.2f} pages/sec")
print(f"  Output Location: {TEMP_PASS1_RAW_DIR}")
print(f"{'='*70}")

timing_data['gpu_inference_pass1'] = gpu_time

# Cell 7: Extract & Crop Figures from Pass 1 Output
- Clears temp/figures/
- Reads each markdown file from temp/pass1_raw/
- Extracts normalized coordinates for images
- Denormalizes to pixel coordinates
- Crops from original page images in temp/pages/
- Saves to temp/figures/ with naming: document_page0001_fig01.png
- Uses NUM_WORKERS for parallel CPU processing

In [ ]:
# Cell 7: Extract & Crop Figures from Pass 1 Output
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger().setLevel(logging.ERROR)

import re
import time
import shutil
from pathlib import Path
from PIL import Image
from concurrent.futures import ThreadPoolExecutor, as_completed

print("="*70)
print("EXTRACTING & CROPPING FIGURES")
print("="*70)

# Clear temp/figures directory
if os.path.exists(TEMP_FIGURES_DIR):
    shutil.rmtree(TEMP_FIGURES_DIR)
os.makedirs(TEMP_FIGURES_DIR, exist_ok=True)

def extract_figure_coordinates(markdown_text):
    """
    Extract ONLY image/figure references with valid bounding boxes.
    Returns list of normalized coordinates (0-999 range).
    """
    figures = []
    lines = markdown_text.split('\n')
    
    for i, line in enumerate(lines):
        # Skip image_caption tags
        if 'image_caption' in line:
            continue
        
        # Look for image tags with coordinates
        if '<|ref|>image<|/ref|>' in line and '<|det|>' in line:
            coord_match = re.search(r'<\|det\|>\[\[(\d+),\s*(\d+),\s*(\d+),\s*(\d+)\]\]<\|/det\|>', line)
            
            if coord_match:
                x1, y1, x2, y2 = map(int, coord_match.groups())
                
                # Validate in normalized space (0-999)
                width = x2 - x1
                height = y2 - y1
                
                # Filter out tiny/invalid boxes
                if width > 50 and height > 50 and x1 < x2 and y1 < y2 and x2 <= 999 and y2 <= 999:
                    figures.append({
                        'bbox_normalized': (x1, y1, x2, y2),
                        'line_number': i
                    })
    
    return figures

def denormalize_bbox(bbox_normalized, image_width, image_height):
    """Convert normalized bounding box (0-999) to pixel coordinates."""
    x1, y1, x2, y2 = bbox_normalized
    
    x1_pixel = int(x1 / 999 * image_width)
    y1_pixel = int(y1 / 999 * image_height)
    x2_pixel = int(x2 / 999 * image_width)
    y2_pixel = int(y2 / 999 * image_height)
    
    return (x1_pixel, y1_pixel, x2_pixel, y2_pixel)

def process_single_page(markdown_path):
    """Process one page: extract figures and crop from original image."""
    try:
        # Read markdown
        with open(markdown_path, 'r', encoding='utf-8') as f:
            markdown_text = f.read()
        
        # Extract figure coordinates
        figures = extract_figure_coordinates(markdown_text)
        
        if not figures:
            return (markdown_path.stem, 0, True, "No figures found")
        
        # Find corresponding page image
        page_stem = markdown_path.stem  # e.g., "document_page0001"
        page_image_path = Path(TEMP_PAGES_DIR) / f"{page_stem}.png"
        
        if not page_image_path.exists():
            return (markdown_path.stem, 0, False, f"Page image not found: {page_image_path}")
        
        # Load original page image
        page_image = Image.open(page_image_path)
        image_width, image_height = page_image.size
        
        # Crop each figure
        figures_cropped = 0
        for fig_idx, fig_info in enumerate(figures, 1):
            bbox_normalized = fig_info['bbox_normalized']
            bbox_pixel = denormalize_bbox(bbox_normalized, image_width, image_height)
            
            # Add padding and crop
            x1, y1, x2, y2 = bbox_pixel
            padding = 10
            x1 = max(0, x1 - padding)
            y1 = max(0, y1 - padding)
            x2 = min(image_width, x2 + padding)
            y2 = min(image_height, y2 + padding)
            
            cropped_figure = page_image.crop((x1, y1, x2, y2))
            
            # Save with naming convention: {page_stem}_fig{01}.png
            fig_filename = f"{page_stem}_fig{fig_idx:02d}.png"
            fig_path = Path(TEMP_FIGURES_DIR) / fig_filename
            cropped_figure.save(fig_path)
            
            figures_cropped += 1
        
        return (markdown_path.stem, figures_cropped, True, None)
        
    except Exception as e:
        return (markdown_path.stem, 0, False, str(e))

# Get all markdown files from pass1_raw
markdown_files = sorted(Path(TEMP_PASS1_RAW_DIR).glob("*.md"))
total_pages = len(markdown_files)

print(f"\nProcessing {total_pages} markdown files")
print(f"Using {NUM_WORKERS} worker threads\n")

extract_start = time.time()

total_figures = 0
successful_pages = 0
failed_pages = 0

with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
    futures = {
        executor.submit(process_single_page, md_path): md_path
        for md_path in markdown_files
    }
    
    for future in as_completed(futures):
        page_stem, num_figures, success, error = future.result()
        
        if success:
            successful_pages += 1
            total_figures += num_figures
            if num_figures > 0:
                print(f"✓ {page_stem}: Extracted {num_figures} figure(s)")
        else:
            failed_pages += 1
            print(f"✗ {page_stem}: {error}")

extract_time = time.time() - extract_start
figures_per_sec = total_figures / extract_time if extract_time > 0 else 0

print(f"\n{'='*70}")
print(f"  Pages Processed: {successful_pages}/{total_pages}")
print(f"  Total Figures Extracted: {total_figures}")
print(f"  Time: {extract_time:.2f}s ({figures_per_sec:.2f} figures/sec)")
print(f"  Workers: {NUM_WORKERS}")
print(f"  Figures saved to: {TEMP_FIGURES_DIR}")
print(f"✓ Figure Extraction Complete")
print(f"{'='*70}")

timing_data['figure_extraction'] = extract_time

# Cell 8a: Pass 2 CPU Preprocessing - Prep Extracted Images
- Reloads config module with Pass 2 settings
- Reloads the image processor to pick up new config
- Loads all extracted figures from temp/figures/
- Applies DeepSeek preprocessing in parallel (using PASS2_NUM_WORKERS)
- Sorts results by filename
- Builds batch_inputs_pass2 (prompt and image tensors) ready for GPU

In [ ]:
# Cell 8a: CPU Preprocessing - Pass 2 Figures
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger().setLevel(logging.ERROR)

from process.image_process import DeepseekOCRProcessor
from PIL import Image
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import sys
import importlib

print("="*70)
print("CPU PREPROCESSING: PASS 2 FIGURES")
print("="*70)

# Force config reload to use Pass 2 settings
print("\n[Config] Reloading config module with Pass 2 settings...")
config_module = sys.modules['config']

# Override config with Pass 2 settings
config_module.BASE_SIZE = PASS2_BASE_SIZE
config_module.IMAGE_SIZE = PASS2_IMAGE_SIZE
config_module.CROP_MODE = PASS2_CROP_MODE
config_module.MIN_CROPS = PASS2_MIN_CROPS
config_module.MAX_CROPS = PASS2_MAX_CROPS
config_module.NUM_WORKERS = PASS2_NUM_WORKERS
config_module.PROMPT = PASS2_PROMPT

# Reload image_process module to pick up new config
if 'process.image_process' in sys.modules:
    import process.image_process
    importlib.reload(process.image_process)

from process.image_process import DeepseekOCRProcessor

# Create new processor instance with Pass 2 settings
processor_pass2 = DeepseekOCRProcessor()

print(f"✓ Processor reloaded with Pass 2 configuration")
print(f"  BASE_SIZE: {config_module.BASE_SIZE}")
print(f"  IMAGE_SIZE: {config_module.IMAGE_SIZE}")
print(f"  CROP_MODE: {config_module.CROP_MODE}")
print(f"  NUM_WORKERS: {config_module.NUM_WORKERS}")

def process_single_figure(fig_path):
    """Apply DeepSeek preprocessing to a single figure"""
    try:
        # Load figure
        figure = Image.open(fig_path)
        
        # Apply DeepSeek preprocessing
        processed = processor_pass2.tokenize_with_images(
            images=[figure], 
            bos=True, 
            eos=True, 
            cropping=config_module.CROP_MODE
        )
        
        return (fig_path.name, True, processed)
    except Exception as e:
        return (fig_path.name, False, str(e))

# Get all figures from temp/figures/ in SORTED order
figure_files = sorted(Path(TEMP_FIGURES_DIR).glob("*.png"))
total_figures = len(figure_files)

if total_figures == 0:
    print("\n⚠ No figures found in temp/figures/")
    print("Run Cell 7 first to extract figures from Pass 1 output")
    batch_inputs_pass2 = []
else:
    print(f"\nFound {total_figures} figures to preprocess")
    print(f"Using {PASS2_NUM_WORKERS} worker threads\n")

    preprocess_start = time.time()

    # Store results with filenames for later sorting
    results = []
    successful = 0
    failed = 0

    with ThreadPoolExecutor(max_workers=PASS2_NUM_WORKERS) as executor:
        # Submit all jobs
        futures = {
            executor.submit(process_single_figure, fig_path): fig_path
            for fig_path in figure_files
        }
        
        # Collect results as they complete
        for future in as_completed(futures):
            fig_name, success, result = future.result()
            
            if success:
                results.append((fig_name, result))
                successful += 1
            else:
                failed += 1
                print(f"✗ Failed: {fig_name}: {result}")
            
            # Progress indicator
            if total_figures > 100 and (successful + failed) % 50 == 0:
                print(f"  Progress: {successful + failed}/{total_figures} figures processed...")

    # CRITICAL: Sort results by filename to maintain order
    results.sort(key=lambda x: x[0])

    # Build batch_inputs_pass2 in sorted order
    batch_inputs_pass2 = []
    for fig_name, processed in results:
        batch_inputs_pass2.append({
            "prompt": PASS2_PROMPT,
            "multi_modal_data": {"image": processed},
        })

    preprocess_time = time.time() - preprocess_start
    figures_per_sec = successful / preprocess_time if preprocess_time > 0 else 0

    print(f"\n{'='*70}")
    print(f"  Total Figures: {total_figures}")
    print(f"  Successful: {successful}")
    print(f"  Failed: {failed}")
    print(f"  Time: {preprocess_time:.2f}s ({figures_per_sec:.2f} figures/sec)")
    print(f"  Workers: {PASS2_NUM_WORKERS}")
    print(f"  Batch inputs ready (SORTED): {len(batch_inputs_pass2)}")
    print(f"✓ Pass 2 Preprocessing Complete")
    print(f"{'='*70}")

    timing_data['deepseek_preprocessing_pass2'] = preprocess_time

# Cell 8b: Run Pass 2 on GPU - "Deep Parsing"
- Creates Pass 2 sampling params (higher temp, skip special tokens)
- Processes all figures through GPU in batches
- Cleans up any grounding artifacts from descriptions
- Stores results in figure_descriptions dictionary (filename → description)
- Keeps everything in memory ready for stitching

In [ ]:
# Cell 8b: GPU Processing - Pass 2 Deep Parse
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger().setLevel(logging.ERROR)

import time
from pathlib import Path
import re

print("="*70)
print("GPU PROCESSING - PASS 2 DEEP PARSE")
print("="*70)

# Check that we have preprocessed figures from Cell 8
if 'batch_inputs_pass2' not in globals() or len(batch_inputs_pass2) == 0:
    print("\n⚠ No figures to process (batch_inputs_pass2 is empty)")
    print("Skipping Pass 2 - no figure descriptions generated")
    figure_descriptions = {}  # Empty dict for downstream cells
    timing_data['gpu_inference_pass2'] = 0.0
    print("\n✓ Pass 2 skipped")
else:
    total_figures = len(batch_inputs_pass2)
print(f"\nProcessing {total_figures} preprocessed figures")
print(f"Batch size (max_num_seqs): {PASS2_MAX_NUM_SEQS}")
print(f"GPU Memory Utilization: {PASS2_GPU_MEMORY_UTIL}")

# Create sampling params for Pass 2
from vllm import SamplingParams
from process.ngram_norepeat import NoRepeatNGramLogitsProcessor

logits_processors_pass2 = [NoRepeatNGramLogitsProcessor(
    ngram_size=20, window_size=50, whitelist_token_ids={128821, 128822}
)]

sampling_params_pass2 = SamplingParams(
    temperature=PASS2_TEMPERATURE,
    max_tokens=PASS2_MAX_TOKENS,
    logits_processors=logits_processors_pass2,
    skip_special_tokens=True,  # Skip grounding tokens for cleaner descriptions
    include_stop_str_in_output=False,
)

print(f"Temperature: {PASS2_TEMPERATURE}")
print(f"Max tokens: {PASS2_MAX_TOKENS}")

# Run GPU inference
gpu_start = time.time()

outputs_pass2 = llm.generate(batch_inputs_pass2, sampling_params=sampling_params_pass2)

gpu_time = time.time() - gpu_start
figures_per_sec = total_figures / gpu_time if gpu_time > 0 else 0

print(f"\n✓ GPU Processing Complete")
print(f"  Time: {gpu_time:.2f}s ({figures_per_sec:.2f} figures/sec)")

# Process outputs and clean up any remaining artifacts
print(f"\nCleaning descriptions...")

figure_files = sorted(Path(TEMP_FIGURES_DIR).glob("*.png"))
figure_descriptions = {}  # filename -> description

for fig_path, output in zip(figure_files, outputs_pass2):
    raw_description = output.outputs[0].text
    
    # Clean any remaining grounding artifacts
    description = raw_description
    description = re.sub(r'<\|ref\|>[^<]*<\|/ref\|>', '', description)
    description = re.sub(r'<\|det\|>[^<]*<\|/det\|>', '', description)
    description = re.sub(r'<\|grounding\|>', '', description)
    description = re.sub(r'<center>.*?</center>', '', description, flags=re.DOTALL)
    description = re.sub(r'<[^>]+>', '', description)
    description = re.sub(r'\n\s*\n\s*\n', '\n\n', description).strip()
    
    figure_descriptions[fig_path.name] = description

print(f"✓ Generated {len(figure_descriptions)} descriptions")

print(f"\n{'='*70}")
print(f"PASS 2 COMPLETE")
print(f"{'='*70}")
print(f"  Total Figures: {total_figures}")
print(f"  GPU Time: {gpu_time:.2f}s")
print(f"  Throughput: {figures_per_sec:.2f} figures/sec")
print(f"  Descriptions stored in memory")
print(f"{'='*70}")

timing_data['gpu_inference_pass2'] = gpu_time

# Cell 9: Merge Text with Image Descriptions and Save Output
- Loads all Pass 1 markdown files in sorted order
- For each page, finds matching figure descriptions by filename pattern
- Inserts descriptions after their corresponding image tags inline
- Adds page numbers and separators
- Combines all pages into one complete document
- Saves to output/{pdf_name}_complete.md
- Handles multiple PDFs

In [ ]:
# Cell 9: Stitch Results & Save Final Output (Multi-PDF)
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger().setLevel(logging.ERROR)

import re
from pathlib import Path
from collections import defaultdict

print("="*70)
print("STITCHING RESULTS & SAVING FINAL OUTPUT")
print("="*70)

# Start timing
stitching_start = time.time() 

# Check that we have figure descriptions from Cell 9
if 'figure_descriptions' not in globals() or len(figure_descriptions) == 0:
    print("\n⚠ No figure descriptions found from Pass 2")
    print("Proceeding with Pass 1 markdown only (no figure descriptions)")
    figure_descriptions = {}

print(f"\nFigure descriptions available: {len(figure_descriptions)}")

def insert_figure_descriptions_in_page(markdown_text, page_stem, figure_descriptions):
    """
    Insert figure descriptions after their corresponding image tags in a single page's markdown.
    """
    if not figure_descriptions:
        return markdown_text
    
    # Find all figures that belong to this page
    # Format: {page_stem}_fig01.png, {page_stem}_fig02.png, etc.
    page_figures = {
        fig_name: desc 
        for fig_name, desc in figure_descriptions.items() 
        if fig_name.startswith(page_stem + "_fig")
    }
    
    if not page_figures:
        return markdown_text
    
    result = markdown_text
    
    # For each figure on this page
    for fig_name, description in sorted(page_figures.items()):
        # Extract figure number from filename (e.g., "document_page0001_fig02.png" -> fig 2)
        fig_match = re.search(r'_fig(\d+)\.png$', fig_name)
        if not fig_match:
            continue
        
        fig_num = int(fig_match.group(1))
        
        # Find the nth image tag in the markdown
        # Pattern: <|ref|>image<|/ref|><|det|>[[...]]<|/det|>
        pattern = r'(<\|ref\|>image<\|/ref\|><\|det\|>\[\[.*?\]\]<\|/det\|>)'
        matches = list(re.finditer(pattern, result))
        
        if fig_num <= len(matches):
            # Get the position of this figure's tag
            match = matches[fig_num - 1]
            tag_end = match.end()
            
            # Insert description after the tag
            description_insert = f'\n\n**Figure {fig_num} Description:**\n{description}\n'
            result = result[:tag_end] + description_insert + result[tag_end:]
    
    return result

# Process all markdown files from Pass 1
markdown_files = sorted(Path(TEMP_PASS1_RAW_DIR).glob("*.md"))
total_pages = len(markdown_files)

print(f"Processing {total_pages} markdown files...")

# Group pages by PDF
# Filename format: {pdf_stem}_page{0001}.md
pdf_pages = defaultdict(list)  # pdf_stem -> list of (page_num, markdown_path)

for md_path in markdown_files:
    page_stem = md_path.stem  # e.g., "document_page0001"
    
    # Extract PDF name and page number
    match = re.match(r'(.+)_page(\d+)$', page_stem)
    if match:
        pdf_stem = match.group(1)
        page_num = int(match.group(2))
        pdf_pages[pdf_stem].append((page_num, md_path))
    else:
        print(f"⚠ Couldn't parse filename: {page_stem}")

print(f"\nFound {len(pdf_pages)} PDF(s) to process\n")

# Process each PDF separately
output_files = []
total_figures_added = 0

for pdf_stem, pages in sorted(pdf_pages.items()):
    print(f"Processing: {pdf_stem}")
    
    # Sort pages by page number
    pages.sort(key=lambda x: x[0])
    
    final_markdown_pages = []
    pages_with_figures = 0
    
    for page_num, md_path in pages:
        page_stem = md_path.stem
        
        # Read original markdown from Pass 1
        with open(md_path, 'r', encoding='utf-8') as f:
            page_markdown = f.read()
        
        # Insert figure descriptions if any exist for this page
        enhanced_markdown = insert_figure_descriptions_in_page(
            page_markdown, 
            page_stem, 
            figure_descriptions
        )
        
        # Check if figures were added
        if enhanced_markdown != page_markdown:
            pages_with_figures += 1
        
        # Add page separator and store
        page_with_header = f"\n{'='*70}\nPAGE {page_num}\n{'='*70}\n\n{enhanced_markdown}\n\n"
        final_markdown_pages.append(page_with_header)
    
    # Combine all pages into final document for this PDF
    final_document = ''.join(final_markdown_pages)
    
    # Save final markdown
    output_filename = f"{pdf_stem}_complete.md"
    output_path = Path(OUTPUT_DIR) / output_filename
    
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(final_document)
    
    output_files.append(output_path)
    total_figures_added += pages_with_figures
    
    print(f"  ✓ {len(pages)} pages, {pages_with_figures} with figures")
    print(f"    Saved: {output_filename} ({output_path.stat().st_size / 1024:.2f} KB)")

print(f"\n{'='*70}")
print(f"{'='*70}")
print(f"  PDFs Processed: {len(pdf_pages)}")
print(f"  Total Pages: {total_pages}")
print(f"  Pages with Figures: {total_figures_added}")
print(f"  Figure Descriptions Added: {len(figure_descriptions)}")
print(f"\n✓ Output files saved to: {OUTPUT_DIR}")
for output_path in output_files:
    print(f"  - {output_path.name}")
print(f"✓ STITCHING COMPLETE")
print(f"{'='*70}")

timing_data['stitching'] = time.time() - stitching_start

# Cell 10: Post-Processing / Markdown Formatting
- Opens each file /workspace/output/
- Adds source filename metadata at top
- Removes all grounding tags and coordinates
- Removes `<center>` tags (keeps the caption text)
- Removes image links
- Removes special tokens
- Cleans up whitespace
- Saves cleaned version with same filename (overwrites)

In [ ]:
# Cell 10: Clean Markdown Output
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger().setLevel(logging.ERROR)

import re
from pathlib import Path

print("="*70)
print("CLEANING MARKDOWN OUTPUT")
print("="*70)

# Start timing
cleaning_start = time.time()

def clean_markdown(text, source_filename):
    """
    Clean markdown by removing grounding tags, coordinates, and special tokens.
    Add source file metadata at the top.
    """
    # Add source file metadata at top
    header = f"<!-- Source: {source_filename} -->\n\n"
    text = header + text
    
    # Remove all grounding tags and their coordinates
    # Pattern: <|ref|>content<|/ref|><|det|>[[coordinates]]<|/det|>
    text = re.sub(r'<\|ref\|>.*?<\|/ref\|><\|det\|>\[\[.*?\]\]<\|/det\|>', '', text)
    
    # Remove any standalone grounding tags that might remain
    text = re.sub(r'<\|ref\|>.*?<\|/ref\|>', '', text)
    text = re.sub(r'<\|det\|>.*?<\|/det\|>', '', text)
    text = re.sub(r'<\|box\|>.*?<\|/box\|>', '', text)
    
    # Remove end-of-sentence tokens
    text = re.sub(r'<｜end▁of▁sentence｜>', '', text)
    
    # Remove image links (markdown image syntax)
    # Pattern: ![anything](./images/anything)
    text = re.sub(r'!\[.*?\]\(\.\/images\/.*?\)', '', text)
    
    # Remove <center> tags but keep the text inside
    # Pattern: <center>text</center>
    text = re.sub(r'<center>(.*?)</center>', r'\1', text)
    
    # Clean up excessive blank lines (3+ blank lines → 2 blank lines)
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    # Remove trailing whitespace from each line
    text = '\n'.join(line.rstrip() for line in text.split('\n'))
    
    # Remove leading/trailing whitespace from entire document
    text = text.strip()
    
    # Ensure file ends with single newline
    text = text + '\n'
    
    return text

# Find all complete markdown files
markdown_files = sorted(Path(OUTPUT_DIR).glob("*_complete.md"))
total_files = len(markdown_files)

if total_files == 0:
    print("\n⚠ No *_complete.md files found in output directory")
    print(f"   Looking in: {OUTPUT_DIR}")
    print("\n✗ Nothing to clean")
else:
    print(f"\nFound {total_files} markdown file(s) to clean\n")
    
    cleaned_files = []
    total_size_before = 0
    total_size_after = 0
    
    for md_path in markdown_files:
        # Extract original PDF filename (remove _complete.md suffix)
        pdf_filename = md_path.stem.replace('_complete', '') + '.pdf'
        
        # Read original markdown
        with open(md_path, 'r', encoding='utf-8') as f:
            original_text = f.read()
        
        original_size = len(original_text)
        total_size_before += original_size
        
        # Clean the markdown
        cleaned_text = clean_markdown(original_text, pdf_filename)
        
        cleaned_size = len(cleaned_text)
        total_size_after += cleaned_size
        
        # Save cleaned version (overwrite original)
        with open(md_path, 'w', encoding='utf-8') as f:
            f.write(cleaned_text)
        
        # Calculate reduction
        reduction = ((original_size - cleaned_size) / original_size * 100) if original_size > 0 else 0
        
        cleaned_files.append(md_path)
        
        print(f"✓ Cleaned: {md_path.name}")
        print(f"  Before: {original_size / 1024:.2f} KB")
        print(f"  After:  {cleaned_size / 1024:.2f} KB")
        print(f"  Reduced: {reduction:.1f}%\n")
    
    # Summary
    print(f"{'='*70}")
    print(f"CLEANING SUMMARY")
    print(f"{'='*70}")
    print(f"  Files Cleaned: {total_files}")
    print(f"  Total Size Before: {total_size_before / 1024:.2f} KB")
    print(f"  Total Size After:  {total_size_after / 1024:.2f} KB")
    
    total_reduction = ((total_size_before - total_size_after) / total_size_before * 100) if total_size_before > 0 else 0
    print(f"  Total Reduction: {total_reduction:.1f}%")
    
    print(f"\n✓ Clean markdown files saved to: {OUTPUT_DIR}")
    for cleaned_path in cleaned_files:
        print(f"  - {cleaned_path.name}")
    
    print(f"\n✓ CLEANING COMPLETE")
    print(f"{'='*70}")

timing_data['markdown_cleaning'] = time.time() - cleaning_start

# Cell 11: Performance Metrics & Benchmarking
- Calculates total pipeline time starting at Cell 6a and stopping here
- Shows detailed timing breakdown with percentages
- Calculates throughput metrics (pages/sec, GB/hour, etc.)
- Shows extrapolations for larger batches
- Summarizes configuration used
- Saves comprehensive JSON report to reports/

In [ ]:
# Cell 11: Performance Metrics & Benchmarking
import time
import json
from pathlib import Path

print("="*70)
print("PERFORMANCE METRICS & BENCHMARKING")
print("="*70)

# Calculate total pipeline time
pipeline_end = time.time()
total_pipeline_time = pipeline_end - pipeline_start

# Get PDF statistics
pdf_files = sorted(Path(PDF_INPUT_DIR).glob("*.pdf"))
total_pdfs = len(pdf_files)
total_size_mb = sum(f.stat().st_size for f in pdf_files) / (1024**2)
total_size_gb = total_size_mb / 1024

# Get page count from temp/pages
page_files = list(Path(TEMP_PAGES_DIR).glob("*.png"))
total_pages = len(page_files)

# Get figure count from temp/figures
figure_files = list(Path(TEMP_FIGURES_DIR).glob("*.png"))
total_figures = len(figure_files)

# Display timing breakdown
print("\n" + "="*70)
print("TIMING BREAKDOWN")
print("="*70)

timing_labels = {
    'pdf_to_images': 'PDF → Page Images (CPU)',
    'deepseek_preprocessing_pass1': 'DeepSeek Preprocessing - Pass 1 (CPU)',
    'gpu_inference_pass1': 'GPU Inference - Pass 1',
    'figure_extraction': 'Figure Extraction & Cropping (CPU)',
    'deepseek_preprocessing_pass2': 'DeepSeek Preprocessing - Pass 2 (CPU)',
    'gpu_inference_pass2': 'GPU Inference - Pass 2',
    'stitching': 'Stitching Results (CPU)'
}

stage_times = []
for key, label in timing_labels.items():
    if key in timing_data:
        stage_time = timing_data[key]
        percentage = (stage_time / total_pipeline_time) * 100 if total_pipeline_time > 0 else 0
        print(f"{label:.<50} {stage_time:>7.2f}s ({percentage:>5.1f}%)")
        stage_times.append(stage_time)
    else:
        print(f"{label:.<50} {'SKIPPED':>7}")

# Calculate overhead (time not accounted for in stages)
accounted_time = sum(stage_times)
overhead = total_pipeline_time - accounted_time
overhead_pct = (overhead / total_pipeline_time) * 100 if total_pipeline_time > 0 else 0

print(f"{'Other/Overhead':.<50} {overhead:>7.2f}s ({overhead_pct:>5.1f}%)")
print(f"{'-'*70}")
print(f"{'TOTAL PIPELINE TIME':.<50} {total_pipeline_time:>7.2f}s (100.0%)")

# Throughput metrics
print("\n" + "="*70)
print("THROUGHPUT METRICS")
print("="*70)

print(f"\nInput:")
print(f"  PDFs Processed: {total_pdfs}")
print(f"  Total Pages: {total_pages}")
print(f"  Total Size: {total_size_mb:.2f} MB ({total_size_gb:.3f} GB)")
print(f"  Figures Extracted: {total_figures}")

print(f"\nSpeed:")
sec_per_page = total_pipeline_time / total_pages if total_pages > 0 else 0
pages_per_sec = total_pages / total_pipeline_time if total_pipeline_time > 0 else 0
print(f"  Seconds/Page: {sec_per_page:.2f}")
print(f"  Pages/Second: {pages_per_sec:.2f}")

if total_figures > 0:
    sec_per_figure = timing_data.get('gpu_inference_pass2', 0) / total_figures
    print(f"  Seconds/Figure (Pass 2): {sec_per_figure:.2f}")

print(f"\nData Throughput:")
if total_size_gb > 0 and total_pipeline_time > 0:
    mb_per_sec = total_size_mb / total_pipeline_time
    gb_per_hour = (total_size_gb / total_pipeline_time) * 3600
    hours_per_gb = (total_pipeline_time / total_size_gb) / 3600
    
    print(f"  MB/second: {mb_per_sec:.2f}")
    print(f"  GB/hour: {gb_per_hour:.2f}")
    print(f"  Hours/GB: {hours_per_gb:.2f}")

# Extrapolations
print(f"\nExtrapolations:")
if total_size_gb > 0:
    time_per_gb = total_pipeline_time / total_size_gb
    print(f"  Time for 1 GB: {time_per_gb/60:.2f} minutes")
    print(f"  Time for 10 GB: {(time_per_gb*10)/60:.2f} minutes ({(time_per_gb*10)/3600:.2f} hours)")
    print(f"  Time for 100 GB: {(time_per_gb*100)/3600:.2f} hours ({(time_per_gb*100)/86400:.2f} days)")

# Configuration summary
print("\n" + "="*70)
print("CONFIGURATION SUMMARY")
print("="*70)
print(f"\nPass 1:")
print(f"  Resolution: {BASE_SIZE}x{BASE_SIZE}, Image Size: {IMAGE_SIZE}")
print(f"  Crop Mode: {CROP_MODE}")
print(f"  Batch Size: {PASS1_MAX_NUM_SEQS}")
print(f"  Temperature: {PASS1_TEMPERATURE}")
print(f"  Max Tokens: {PASS1_MAX_TOKENS}")

print(f"\nPass 2:")
print(f"  Resolution: {PASS2_BASE_SIZE}x{PASS2_BASE_SIZE}, Image Size: {PASS2_IMAGE_SIZE}")
print(f"  Crop Mode: {PASS2_CROP_MODE}")
print(f"  Batch Size: {PASS2_MAX_NUM_SEQS}")
print(f"  Temperature: {PASS2_TEMPERATURE}")
print(f"  Max Tokens: {PASS2_MAX_TOKENS}")

print(f"\nCPU:")
print(f"  Workers (Pass 1): {NUM_WORKERS}")
print(f"  Workers (Pass 2): {PASS2_NUM_WORKERS}")

print(f"\nGPU:")
print(f"  Model: {torch.cuda.get_device_name(0)}")
print(f"  Memory Utilization: {PASS1_GPU_MEMORY_UTIL}")

# Save detailed JSON report
timestamp = int(time.time())
report_data = {
    "timestamp": timestamp,
    "pipeline": {
        "total_time_seconds": total_pipeline_time,
        "total_time_minutes": total_pipeline_time / 60,
        "total_time_hours": total_pipeline_time / 3600
    },
    "input": {
        "pdfs_processed": total_pdfs,
        "total_pages": total_pages,
        "total_figures": total_figures,
        "total_size_mb": total_size_mb,
        "total_size_gb": total_size_gb
    },
    "timing_breakdown": timing_data,
    "throughput": {
        "seconds_per_page": sec_per_page,
        "pages_per_second": pages_per_sec,
        "mb_per_second": mb_per_sec if total_size_gb > 0 else 0,
        "gb_per_hour": gb_per_hour if total_size_gb > 0 else 0,
        "hours_per_gb": hours_per_gb if total_size_gb > 0 else 0
    },
    "configuration": {
        "pass1": {
            "base_size": BASE_SIZE,
            "image_size": IMAGE_SIZE,
            "crop_mode": CROP_MODE,
            "batch_size": PASS1_MAX_NUM_SEQS,
            "temperature": PASS1_TEMPERATURE,
            "max_tokens": PASS1_MAX_TOKENS,
            "prompt": PASS1_PROMPT
        },
        "pass2": {
            "base_size": PASS2_BASE_SIZE,
            "image_size": PASS2_IMAGE_SIZE,
            "crop_mode": PASS2_CROP_MODE,
            "batch_size": PASS2_MAX_NUM_SEQS,
            "temperature": PASS2_TEMPERATURE,
            "max_tokens": PASS2_MAX_TOKENS,
            "prompt": PASS2_PROMPT
        },
        "cpu_workers_pass1": NUM_WORKERS,
        "cpu_workers_pass2": PASS2_NUM_WORKERS,
        "gpu": {
            "device": torch.cuda.get_device_name(0),
            "memory_utilization": PASS1_GPU_MEMORY_UTIL
        }
    },
    "pdfs": [
        {
            "filename": pdf.name,
            "size_mb": pdf.stat().st_size / (1024**2)
        }
        for pdf in pdf_files
    ]
}

report_filename = f"benchmark_report_{timestamp}.json"
report_path = Path(REPORTS_DIR) / report_filename

with open(report_path, 'w', encoding='utf-8') as f:
    json.dump(report_data, f, indent=2)

print("\n" + "="*70)
print(f"✓ Benchmark report saved: {report_filename}")
print(f"  Location: {report_path}")
print("="*70)

# Reference - Resolution Settings

For Tiny mode:
- BASE_SIZE = 512
- IMAGE_SIZE = 512
- CROP_MODE = False

For Small mode:
- BASE_SIZE = 640
- IMAGE_SIZE = 640
- CROP_MODE = False

For Base mode:
- BASE_SIZE = 1024
- IMAGE_SIZE = 1024
- CROP_MODE = False

For Large mode:
- BASE_SIZE = 1280
- IMAGE_SIZE = 1280
- CROP_MODE = False

For Gundam mode (default):
- BASE_SIZE = 1024
- IMAGE_SIZE = 640
- CROP_MODE = True  # This is the key to enabling/disabling Gundam mode!